In [6]:
import requests
from bs4 import BeautifulSoup as bs
import time
import csv

def get_car_links(main_url, page_number):
    try:
        url = f"{main_url}-p{page_number}"
        response = requests.get(url, headers={'User-agent': 'Mozilla/5.0'})
        response.raise_for_status()

        soup = bs(response.text, 'html.parser')
        car_links = [a['href'] for a in soup.select('a.inventory')]
        return car_links
    except requests.exceptions.RequestException as e:
        print(f"Erreur lors de la récupération des liens de la page {page_number} : {e}")
        return []

def scrape_car_details(car_url):
    try:
        response = requests.get(car_url, headers={'User-agent': 'Mozilla/5.0'})
        response.raise_for_status()

        car_soup = bs(response.text, 'html.parser')

        # Obtenez le nom de la voiture depuis la balise h2
        car_name_element = car_soup.find('div', class_='col-lg-12 col-md-12 col-sm-12 col-xs-12').find('h2')
        car_name = car_name_element.text.strip() if car_name_element else "Nom non trouvé"

        # Obtenez les détails de la voiture
        car_details_elements = car_soup.find('table', class_='table table-striped table-bordered')
        car_details = ' '.join(detail.text.strip() for detail in car_details_elements.find_all('td')) if car_details_elements else "Details non trouvé"

        # Obtenez le prix de la voiture
        car_price_element = car_soup.find(class_='col-lg-4 col-md-4 col-sm-4 text-right')
        car_price = car_price_element.text.strip() if car_price_element else "Prix non trouvé"

        # Processus des données récupérées
        print(f"Nom de la voiture: {car_name}")
        print(f"Details de la voiture: {car_details}")
        print(f"Prix de la voiture: {car_price}\n{'='*30}")

        return {'Car Name': car_name, 'Car Details': car_details, 'Car Price': car_price}

    except requests.exceptions.RequestException as e:
        print(f"Erreur lors de la récupération des détails de la voiture : {e}")
        return None


def scrap_pages(main_url, start_page, end_page):
    car_details_list = []

    for page_number in range(start_page, end_page + 1):
        car_links = get_car_links(main_url, page_number)

        for car_link in car_links:
            print(f"Scrapping details for car: {car_link}")
            car_detail = scrape_car_details(car_link)
            
            if car_detail:
                car_details_list.append(car_detail)
            else:
                print(f"Aucun détail trouvé pour la voiture : {car_link}")
            
            time.sleep(2)  # Respectez les politiques du site pour éviter d'être bloqué

    return car_details_list

main_page_url = 'https://www.usedcarsgroup.com/used-cars-for-sale'
total_pages = 1000  # Modifier le nombre total de pages selon les besoins
start_page = 1
end_page = total_pages
car_details_list = scrap_pages(main_page_url, start_page, end_page)

# Ajout de débogage
print("Nombre total de détails récupérés :", len(car_details_list))

# Écrire les détails des voitures dans un fichier CSV avec espace entre chaque détail
with open('car_details_usedcargroup.csv', 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['Car Name', 'Car Details', 'Car Price']  # Ajoutez 'Car Name' à la liste
    csv_writer = csv.DictWriter(csvfile, fieldnames=fieldnames, delimiter=' ')
    csv_writer.writeheader()
    
    for detail in car_details_list:
        csv_writer.writerow(detail)


Scrapping details for car: https://www.usedcarsgroup.com/used-2017-volkswagen-jetta-burlington-nj-3vwb67ajxhm298148
Nom de la voiture: 2017 VOLKSWAGEN JETTA SE - 3VWB67AJXHM298148
Details de la voiture: Asking price $15,495 near Burlington, NJ 08016 miles driven 72,454 mi avg. miles 10,350 mi/yr condition Used body style Sedan engine 4 Cyl transmission Manual drivetrain FWD exterior color Blue stock # 298148 vin # 3VWB67AJXHM298148 History Vehicle Report avg mileage 33 mpg fuel type Gasoline features Full Options List
Prix de la voiture: $ 17,044
$ 15,495

$277.73 Est. Mo.
Scrapping details for car: https://www.usedcarsgroup.com/used-2021-kia-telluride-san_angelo-tx-5xyp54hc1mg136546
Nom de la voiture: 2021 KIA TELLURIDE SX - 5XYP54HC1MG136546
Details de la voiture: Asking price $39,390 near San Angelo, TX 76903 miles driven 92,687 mi avg. miles 30,895 mi/yr condition Used body style SUV engine 6 Cyl transmission Automatic drivetrain FWD exterior color Black interior color Black stock 

In [22]:
import time
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.service import Service as FirefoxService
from selenium.webdriver.firefox.options import Options as FirefoxOptions

def get_car_links(main_url, page_number):
    try:
        # Initialiser le navigateur web (Mozilla Firefox)
        firefox_path = "C:\Program Files\Mozilla Firefox\firefox.exe" # Remplacez cela par le chemin correct
        options = FirefoxOptions()
        options.headless = True
        service = FirefoxService(executable_path=firefox_path)
        driver = webdriver.Firefox(service=service, options=options)

        # Charger la page
        url = f"{main_url}?firstRecord={25 * (page_number - 1)}&isNewSearch=false"
        driver.get(url)

        # Attendre que la page se charge complètement
        time.sleep(10)  # Augmentez le temps d'attente si nécessaire

        # Extraire tous les liens de la page
        car_links = [a.get_attribute('href') for a in driver.find_elements(By.CSS_SELECTOR, 'a')]
        
        # Filtrer les liens qui ne sont pas pertinents (ajuster selon la structure de la page)
        car_links = [link for link in car_links if 'vehicle-details' in link]

        # Fermer le navigateur
        driver.quit()

        # Afficher les liens pour débogage
        print(car_links)

        return car_links
    except Exception as e:
        print(f"Erreur lors de la récupération des liens de la page {page_number} : {e}")
        return []

def scrape_car_details(car_url):
    try:
        # Initialiser le navigateur web (Mozilla Firefox)
        firefox_path = "C:\Program Files\Mozilla Firefox\firefox.exe"
        options = FirefoxOptions()
        options.headless = True
        service = FirefoxService(executable_path=firefox_path)
        driver = webdriver.Firefox(service=service, options=options)

        # Charger la page de détails de la voiture
        driver.get(car_url)

        # Attendre que la page se charge complètement
        time.sleep(5)  # Augmentez le temps d'attente si nécessaire

        # Extraire les informations directement avec Selenium
        car_name_element = driver.find_element(By.CSS_SELECTOR, 'h1.text-ultra-bold.text-size-400.text-size-sm-700.col-xs-12.col-sm-7.col-md-8')
        car_name = car_name_element.text.strip() if car_name_element else "Nom non trouvé"

        car_details_elements = driver.find_element(By.CSS_SELECTOR, 'table.accordion.accordion-filters.text-left')
        car_details = ' '.join(detail.text.strip() for detail in car_details_elements.find_elements(By.CSS_SELECTOR, 'td')) if car_details_elements else "Details non trouvé"

        car_price_element = driver.find_element(By.CLASS_NAME, 'firstPrice')
        car_price = car_price_element.text.strip() if car_price_element else "Prix non trouvé"

        # Processus des données récupérées
        print(f"Nom de la voiture: {car_name}")
        print(f"Details de la voiture: {car_details}")
        print(f"Prix de la voiture: {car_price}\n{'='*30}")

        # Fermer le navigateur
        driver.quit()

        return {'Car Name': car_name, 'Car Details': car_details, 'Car Price': car_price}

    except Exception as e:
        print(f"Erreur lors de la récupération des détails de la voiture : {e}")
        return None

def scrap_pages(main_url, total_pages):
    car_details_list = []

    for page_number in range(1, total_pages + 1):
        start_record = (page_number - 1) * 25 + 1
        car_links = get_car_links(main_url, start_record)

        for car_link in car_links:
            print(f"Scrapping details for car: {car_link}")
            car_detail = scrape_car_details(car_link)
            
            if car_detail:
                car_details_list.append(car_detail)
            else:
                print(f"Aucun détail trouvé pour la voiture : {car_link}")
            
            time.sleep(2)  # Respectez les politiques du site pour éviter d'être bloqué

    return car_details_list

main_url = 'https://www.autotrader.com/cars-for-sale/used-cars'
total_pages = 2  # Modifier le nombre total de pages selon les besoins
car_details_list = scrap_pages(main_url, total_pages)

# Ajout de débogage
print("Nombre total de détails récupérés :", len(car_details_list))

# Écrire les détails des voitures dans un fichier CSV avec espace entre chaque détail
with open('car_details_autotrader.csv', 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['Car Name', 'Car Details', 'Car Price']  # Ajoutez 'Car Name' à la liste
    csv_writer = csv.DictWriter(csvfile, fieldnames=fieldnames, delimiter=' ')
    csv_writer.writeheader()
    
    for detail in car_details_list:
        csv_writer.writerow(detail)


Erreur lors de la récupération des liens de la page 1 : Message: Unable to locate or obtain driver for firefox; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors/driver_location

Erreur lors de la récupération des liens de la page 26 : Message: Unable to locate or obtain driver for firefox; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors/driver_location

Nombre total de détails récupérés : 0
